In [1]:
import nltk
import json
import spacy
nltk.download('stopwords')


In [12]:
from nltk.corpus import stopwords

In [18]:
nlp = spacy.load("en_core_web_lg")

In [50]:
from pyemd import emd
from gensim.similarities import WmdSimilarity

In [48]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# you need to install gensim and download the glove.6b.200d.txt online

# inputs are two sentence, s1 is the question, s2 is the sentence from the text
# the output is the distance between two sentences. When the number is smaller, the two sentences are more similar.
def calculate_distance(s1, s2):
    glove_input_file = 'glove.6B.200d.txt'
    word2vec_output_file = 'glove.6B.200d.txt.word2vec'
    glove2word2vec(glove_input_file, word2vec_output_file)

    # remove stopwords
    s1 = remove_stopwords(s1)
    s2 = remove_stopwords(s2)

    # load the Stanford GloVe model
    filename = 'glove.6B.200d.txt.word2vec'
    model = KeyedVectors.load_word2vec_format(filename, binary=False)

    # calculate distance between two sentences using WMD algorithm
    distance = model.wmdistance(s1, s2)

    return distance


def remove_stopwords(s):
    # remove stopwords
    stopwords = nltk.corpus.stopwords.words("english")

    word_tokens = word_tokenize(s)
    filtered_sentence = [w.strip() for w in word_tokens if w not in stopwords]
    text = " ".join(str(x) for x in filtered_sentence)
    return text


In [68]:
paragraph = "In November 2006, the Victorian Legislative Council elections were held under a new multi-member proportional representation system. The State of Victoria was divided into eight electorates with each electorate represented by five representatives elected by Single Transferable Vote. The total number of upper house members was reduced from 44 to 40 and their term of office is now the same as the lower house members—four years. Elections for the Victorian Parliament are now fixed and occur in November every four years. Prior to the 2006 election, the Legislative Council consisted of 44 members elected to eight-year terms from 22 two-member electorates."

In [4]:
def parse_sentences(paragraph): 
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sent_detector.tokenize(paragraph.strip())
    return sentences

In [92]:
sen = parse_sentences(paragraph)

In [190]:
# answer_sen = ''
# max_score = 0
# question = 'How many campuses does the University of California have?'
# for sen in parse_sentences(paragraph):
#     new_score = calculate_distance(sen, question)
#     if new_score > maxscore:
#         maxscore = new_score
#         answer_sen = sen

In [194]:
q = "How many metropolitan areas does Southern California's population encompass?"
sentence = "Southern California's population encompasses seven metropolitan areas, or MSAs: the Los Angeles metropolitan area, consisting of Los Angeles and Orange counties; the Inland Empire, consisting of Riverside and San Bernardino counties"

In [180]:
def get_entity(sentences):
    entity = []
    doc = nlp(sentences)
    for ent in doc.ents:
        entity.append( (ent.text,ent.label_))
    return entity

In [181]:
#find cardinal entity
#input: sentence
def locate_cardinal(sentence):
    cardinal = []
    for en in get_entity(sentence):
        if en[1] == 'CARDINAL':
            cardinal.append(en[0])
    return cardinal

In [138]:
def get_lemma(word):
    token = nlp(word)
    lemma = token[0].lemma_
    return lemma


In [184]:
def calculate_word_distance(lemma, card, sentence):
    sentence_token = nlp(sentence)
    idx = 0
    obj_idx = 0
    card_idx = 0
    for s in sentence_token:
        #print(s.text)
        if s.lemma_ == lemma:
            obj_idx = idx
        if s.text == card:
            card_idx = idx
        idx += 1
    if card_idx - obj_idx == 0:
        return 9999
    else:
        return abs(card_idx - obj_idx)

In [186]:
def answer_how(question, sentence):
    noun = get_noun(question)
    noun_lemma = get_lemma(noun)
    card_list = locate_cardinal(sentence)
    min_dist = 9999
    answer = ''
    for card in card_list:
        #print(noun_lemma)
        dist = calculate_word_distance(noun_lemma, card, sentence)
        if dist < min_dist:
            mind_dist = dist
            answer = card
    return answer

In [134]:
def get_noun(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == 'NOUN':
            return token.text

In [195]:
answer_how(q, sentence)

'seven'